In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)
import seaborn as sns
from datetime import date

In [ ]:
# Data from https://www.ncsbe.gov/results-data/voter-registration-data
# Downloaded 11/7/25

## Load the Watauga and NC datasets

In [ ]:
# Loading the voter registration history

file = 'ncvhis95.txt'

turn_df = pd.read_csv(
    file,
    delimiter="\t",        
    usecols = ['voted_party_cd', 'ncid', 'election_lbl'],
    encoding="latin-1",  
)

turn_df.head()

In [ ]:
# Loading the voter registration count


file = 'ncvoter95.txt'

wat_df = pd.read_csv(
    file,
    delimiter="\t",        
    usecols = ['zip_code', 'registr_dt', 'race_code', 'ethnic_code', 'party_cd', 'gender_code', 'birth_year', 'status_cd', 'precinct_desc', 'ncid'],
    encoding="latin-1",  
)


wat_df.head()

In [ ]:
merged = pd.merge(turn_df, wat_df, how = 'right', on = 'ncid', indicator = True)
merged.head(5)

In [ ]:
merged = merged[merged['registr_dt'] != '##/##/####']
merged['registr_dt'] = pd.to_datetime(merged['registr_dt'])
merged['election_lbl'] = pd.to_datetime(merged['election_lbl'])
merged = merged.set_index('registr_dt')

merged_asc = merged.sort_index(ascending=True)
merged_asc.head()

In [ ]:
file = 'ncvoter_Statewide.txt'

nc_df = pd.read_csv(
    file,
    delimiter="\t",        
    usecols = ['zip_code', 'registr_dt', 'race_code', 'ethnic_code', 'party_cd', 'gender_code', 'birth_year', 'status_cd', 'precinct_desc', 'ncid'],
    encoding="latin-1",   
)

nc_df = nc_df[nc_df['status_cd'] == 'A']
nc_df.count()

In [ ]:
nc_df['precinct_desc'].unique()

In [ ]:
nc_na = nc_df['precinct_desc'].isna() 
nc_empty = (nc_df['precinct_desc'] == '').sum()
print(nc_na.sum())
print(nc_empty.sum())

## Clean and filter Watauga dataset

In [ ]:
wat_df = wat_df[wat_df['registr_dt'] != '##/##/####']
wat_df['registr_dt'] = pd.to_datetime(wat_df['registr_dt'])
wat_df = wat_df.set_index('registr_dt')

wat_df_asc = wat_df.sort_index(ascending=True)

In [ ]:
wat_df_asc.head()

In [ ]:
wat_current = wat_df_asc.loc['2025-08-01':]
wat_past = wat_df_asc.loc['2023-08-01':'2023-11-07']
percent = (len(wat_past) - len(wat_current)) / len(wat_past)
print(f'The 2023 Watauga election had a registration count of {len(wat_past)}, while the 2025 election cycle ' +
      f'had a registration count of {len(wat_current)}; a {percent:.1%} decrease.')

## Clean and filter NC dataset

In [ ]:
nc_df = nc_df[nc_df['registr_dt'] != '##/##/####']
nc_df['registr_dt'] = pd.to_datetime(nc_df['registr_dt'])
nc_df = nc_df.set_index('registr_dt')

nc_df_asc = nc_df.sort_index(ascending=True)

In [ ]:
nc_current = nc_df_asc.loc['2025-08-01':]
nc_past = nc_df_asc.loc['2023-08-01':'2023-11-07']
percent = (len(nc_past) - len(nc_current)) / len(nc_past)
print(f'The 2023 NC election had a new registration count of {len(nc_past)}, while the 2025 election cycle ' +
      f'had a registration count of {len(nc_current)}; a {percent:.1%} decrease.')

## Group-by-precinct Watauga Dataset

In [ ]:
# wat_current.groupby(['precinct_desc','race_code']).size().unstack('race_code').plot(kind = 'bar', ylim=(0, 90), stacked = True, figsize=(10, 6))


In [ ]:
merged_asc.groupby(['precinct_desc', 'election_lbl'], dropna = False)['ncid'].count().unstack('election_lbl').iloc[:, -13:]



## Plots!

In [ ]:
# fig, axes = plt.subplots(2, 2, figsize = (10, 6))

# nc_past['party_cd'].value_counts().plot.bar(ax = axes[0,0], ylim=(0, 70000), color = ['grey', 'red', 'blue', 'green'], title = '2023 NC Party', rot = 0)
# nc_current['party_cd'].value_counts().plot.bar(ax = axes[0,1], ylim=(0, 70000), color = ['grey', 'red', 'blue', 'green'], title = '2025 NC Party', rot = 0)

# wat_past['party_cd'].value_counts().plot.bar(ax = axes[1,0], ylim=(0, 1200), color = ['grey', 'blue', 'red', 'green'], title = '2023 WAT Party', rot = 0)
# wat_current['party_cd'].value_counts().plot.bar(ax = axes[1,1], ylim=(0, 1200), color = ['grey', 'blue', 'red', 'green'], title = '2023 WAT Party', rot = 0)

# plt.tight_layout(h_pad=3.0) 


In [ ]:
# fig, axes = plt.subplots(2, 2, figsize = (10, 6))

# nc_past['gender_code'].value_counts().plot.bar(ax = axes[0,0], ylim=(0, 61000), color = ['hotpink', 'blue', 'grey'], title = '2023 NC Gender', rot = 0)
# nc_current['gender_code'].value_counts().plot.bar(ax = axes[0,1], ylim=(0, 61000), color = ['hotpink', 'blue', 'grey'], title = '2025 NC Gender', rot = 0)

# wat_past['gender_code'].value_counts().plot.bar(ax = axes[1,0], ylim=(0, 1100), color = ['hotpink', 'blue', 'grey'], title = '2023 WAT Gender', rot = 0)
# wat_current['gender_code'].value_counts().plot.bar(ax = axes[1,1], ylim=(0, 1100), color = ['hotpink', 'blue', 'grey'], title = '2025 WAT Gender', rot = 0)

# plt.tight_layout(h_pad=3.0) 


In [ ]:
fig, axes = plt.subplots(2, 2, figsize = (10, 6))

nc_past['race_code'].value_counts().plot.bar(ax = axes[0,0], ylim=(0, 80000), color = ['whitesmoke', 'black', 'grey', 'green'], edgecolor='black', title = '2023 NC Race', rot = 0)
nc_current['race_code'].value_counts().plot.bar(ax = axes[0,1], ylim=(0, 80000), color = ['whitesmoke', 'grey', 'black', 'green'], edgecolor='black', title = '2025 NC Race', rot = 0)

wat_past['race_code'].value_counts().plot.bar(ax = axes[1,0], ylim=(0, 1600), color = ['whitesmoke', 'grey', 'black', 'green'], edgecolor='black', title = '2023 WAT Race', rot = 0)
wat_current['race_code'].value_counts().plot.bar(ax = axes[1,1], ylim=(0, 1600), color = ['whitesmoke', 'grey', 'black', 'green'], edgecolor='black', title = '2025 WAT Race', rot = 0)

plt.tight_layout(h_pad=3.0) 


In [ ]:
# Explore UNA data

una_nc_past = nc_past[nc_past['party_cd'] == 'UNA']
una_nc_current = nc_current[nc_current['party_cd'] == 'UNA']

una_wat_past = wat_past[wat_past['party_cd'] == 'UNA']
una_wat_current = wat_current[wat_current['party_cd'] == 'UNA']


## Explore UNA demographics

## Explore DEM demographics

## Function time!!

In [ ]:
nc_current = nc_df_asc.loc['2025-08-01':]
nc_past = nc_df_asc.loc['2023-08-01':'2023-11-07']
percent = (len(nc_past) - len(nc_current)) / len(nc_past)
print(f'The 2023 NC election had a new registration count of {len(nc_past)}, while the 2025 election cycle ' +
      f'had a registration count of {len(nc_current)}; a {percent:.1%} decrease.')

In [ ]:
def percent_chg_df(df, group, col, past1, past2, current1, current2):
    
    # Filter df by date range desired
    past = df.loc[past1:past2]
    current = df.loc[current1:current2]
    
    past_df = pd.DataFrame(past.groupby([f'{group}',f'{col}']).size().unstack(f'{col}'))
    current_df = pd.DataFrame(current.groupby([f'{group}',f'{col}']).size().unstack(f'{col}'))

    # past_df = past_df.T

    pct_chg_df = ((past_df - current_df) / past_df) * 100
    
    ax = sns.heatmap(pct_chg_df, annot=True, cmap='crest', fmt=".1f", linewidths=.5)

    ax.xaxis.tick_top()
    ax.set(xlabel="", ylabel="")

    return ax
    
   # return round(pct_chg_df, 1)

In [ ]:
percent_chg_df(wat_df_asc, 'gender_code', 'party_cd', date(2023, 8, 1), date(2023, 11, 7), date(2025, 8, 1), date(2025, 11, 7))



In [ ]:
# Look at pct of inactive voters in each precinct
# 

## Cons
- None of these figures have been fact checked
    - We need to compare all this to Corissa's Google Sheets and make sure both the absolute figures and percentages agree
- We haven't established the specific time periods we are wanting to compare, so my numbers will be off if the timeline changes
- I have not filtered out dead people or incorporated the absentee data, so my numbers are incomplete.

## Pros
- We are not starting from scratch, given the above code. My numbers are incomplete and wrong right now, but, depending on what analyses we are aiming for, the above code can be modified appropriately.

## Looking forward
- I think we need to establish what we are looking to achieve out of all this data analysis to form the questions we are want to answer.
- We could look at projecting what percent of every 100 new registered voters would vote Democrat to show why putting more effort into canvassing, etc. is extremely important if Democrats want to win. More VRs = more wins!!
- Percent UNA that was previously Democrat.
- How do we answer a question like where should we put out limited resources to net the most VRs?